In [6]:
import torchvision
import torch.nn as nn
import torch
import json
from PIL import Image
from torchvision.transforms import Resize
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
import torchvision.transforms as T
from tqdm.notebook import tqdm

In [7]:

class TrainingDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.images = []
        self.boxes = []



        x_final, y_final = 600, 1200
        transform = Resize((x_final, y_final))
        to_tensor = ToTensor()
        for item in data:

            file_name = item['data']['image'].split('-')[-1]

            # r'C:\Users\Boows\Desktop\sieci\\' +
            image_path = file_name  # Zmień odpowiednio ścieżkę do katalogu ze zdjęciami
            image = Image.open(image_path)
            image = transform(to_tensor(image))
            self.images.append(image)

            points = item['annotations'][0]['result'][0]['value']['points']

            xmin = min(point[0] for point in points)/100*x_final
            ymin = min(point[1] for point in points)/100*y_final
            xmax = max(point[0] for point in points)/100*x_final
            ymax = max(point[1] for point in points)/100*y_final
            box = torch.tensor([[xmin, ymin, xmax, ymax]])
            self.boxes.append(box)

        self.labels = torch.zeros((len(data), 1), dtype=torch.int64)


        self.transforms = T.Compose([
        T.RandomApply(T.CenterCrop(10),p=0.5),
        T.RandomApply(T.PILToTensor(),p=0.5),
        T.RandomApply(T.ElasticTransform(alpha=45.0),p=0.5),
        T.RandomApply(T.ConvertImageDtype(torch.float),p=0.5)])
        self.transforms = None

    # def crop_img_with_box(self, img, box):
    #     # TODO: crop image with box
    #     return img, box

    def __getitem__(self, idx):
        out_img = self.images[idx]
        if self.transforms is not None:
            out_img = self.transforms(out_img)

        out_box = self.boxes[idx]
        # out_img, out_box = self.crop_img_with_box(out_img, out_box)

        return out_img, out_box, self.labels[idx]

    def __len__(self):
        return len(self.images)





In [8]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn()
model.roi_heads.box_predictor.cls_score = nn.Linear(1024, 1, bias=True)
model.roi_heads.box_predictor.bbox_pred = nn.Linear(1024, 4, bias=True)

#Opcjonalne zamrożenie wag
weights_dict = dict(model.named_parameters())
for k, v in weights_dict.items():
    if "box_predictor" not in k:
        v.requires_grad = False

In [9]:
# Wczytaj plik JSON
with open('labelj.json') as f:
    data = json.load(f)

In [10]:
#Hiperparametr
epochs = 10
#lr, momentum - hiperparametry
#można spróbować torch.optim.Adam
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
training_dataset = TrainingDataset(data)
#batch_size = 2 - hiperparametr
training_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=2, shuffle=True)
val_dataset = TrainingDataset(data)
val_dataloader = torch.utils.data.DataLoader(training_dataset, batch_size=2, shuffle=True)

for epoch in tqdm(list(range(epochs))):
    for images, boxes, labels in training_dataloader:
        images = list(image.to(device) for image in images)
        targets = []
        for i in range(len(images)):
            d = {}
            d['boxes'] = boxes[i].to(device)
            d['labels'] = labels[i].to(device)
            targets.append(d)
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        losses.backward()
        optimizer.step()
        optimizer.zero_grad()
        print(losses)

    for images, boxes, labels in val_dataloader:
        images = list(image.to(device) for image in images)
        targets = []
        for i in range(len(images)):
            d = {}
            d['boxes'] = boxes[i].to(device)
            d['labels'] = labels[i].to(device)
            targets.append(d)
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        print(f"Val loss: {losses.item()}")

#Zapisanie modelu
torch.save(model.state_dict(), "model.pth")


ImportError: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html